# 溯源树-web搜索版
1. 接收一个论文的链接、或pdf、或论文名，然后：
    * 如果是pdf，直接使用pypdf解析
    * 如果是链接，获取其pdf版本，
    * 如果是论文名，使用google scholar搜索，获取其pdf链接，然后使用pypdf解析
2. LLM解析论文，获取{title，publication_date，abstract，……，category，references[article_1,...]} 
3. 在谷歌学术上搜索所有引用论文的论文名，回到1.的接收论文名的情况，对这些论文也进行解析，然后基于一定的方法（比如论文引用数、摘要的相似度）选取其中的top k，这种做法可以只对摘要进行，因为嵌入模型处理长度有上限

## 1.由论文名获取论文内容
- pdf解析  
- 论文名搜索-测试api调用时是否有websearch能力？--没有
- 没有的话就得结合工具了，看一下能不能直接获取论文内容

In [ ]:
# 依赖安装
! pip install pydantic
! pip install langchain
! pip install langchain_community
! pip install PyPDF2
! pip install anytree
! pip install langchain_core
! pip install

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [1]:
import os
from dotenv import load_dotenv

# 加载 .env 文件
load_dotenv()

# 尝试获取环境变量并打印出来
dashscope_api_key = os.getenv("DASHSCOPE_API_KEY")

In [2]:
from langchain_community.llms import Tongyi
from langchain.prompts import PromptTemplate

通过arxiv，根据论文名搜索论文原文

In [23]:
import requests
from PyPDF2 import PdfReader
from io import BytesIO
import re
from xml.etree import ElementTree as ET


def search_paper(paper_title):
    """
    搜索论文并返回其文本内容。
    
    参数:
        paper_title (str): 论文标题
        
    返回:
        str: 论文的文本内容（如果成功解析），否则返回 None
    """
    def clean_title(title):
        # 移除非字母数字字符，并转换为小写
        return re.sub(r'[^\w\s\-]', '', title).strip().lower()

    paper_title = clean_title(paper_title)
    
    # Step 1: 在arXiv上搜索论文并获取PDF链接
    url = "http://export.arxiv.org/api/query"
    
    # 参数设置：搜索标题中包含的关键词
    params = {
        'search_query': f'ti:"{paper_title}"',  # ti: 表示按标题搜索
        'start': 0,
        'max_results': 1  # 只返回第一个结果
    }
    
    # 寻找最早的论文
    params['sort_by'] = 'submittedDate'
    params['sort_order'] = 'ascending'
    
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print(f"请求失败，状态码: {response.status_code}")
        return None
    
    # 解析arXiv返回的XML数据
    root = ET.fromstring(response.content)
    namespace = {'atom': 'http://www.w3.org/2005/Atom'}
    entry = root.find('atom:entry', namespace)
    
    if entry is None:
        print("未找到相关论文")
        return None
    
    # 提取论文的PDF链接
    pdf_link = entry.find('atom:id', namespace).text.replace('abs', 'pdf') + '.pdf'
    
    # Step 2: 使用PyPDF2解析PDF文件（不下载PDF）
    response = requests.get(pdf_link)
    
    if response.status_code != 200:
        print(f"无法获取PDF内容，状态码: {response.status_code}")
        return None
    
    # 将二进制内容转换为BytesIO对象
    pdf_content = BytesIO(response.content)
    
    # 使用PyPDF2解析PDF内容
    reader = PdfReader(pdf_content)
    number_of_pages = len(reader.pages)
    text = ""
    
    for page in range(number_of_pages):
        page_text = reader.pages[page].extract_text()
        if page_text:
            text += page_text
    
    if text.strip():
        return text
    else:
        print("无法提取论文文本内容")
        return None

In [4]:
# 测试
# paper_title = input("请输入论文标题: ")
    
# paper_text = search_paper(paper_title)
    
# if paper_text:
#     print("\n论文内容解析如下:\n")
#     print(paper_text[:500])  # 打印前500个字符作为示例
# else:
#     print("未能获取论文文本")

In [ ]:
# TODO：查找成功率较低，之后探索如何解析为容易查找到的格式、查找时应该使用什么样的param
# paper_text = search_paper("HotpotQA")
# paper_text

## 2.解析论文LLM
和之前差不多，解析论文信息：题目、发布时间、摘要、引用文献；  

In [15]:
from langchain_community.llms import Tongyi
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class PaperInfo(BaseModel):
    """
    定义论文信息的结构化输出格式。
    """
    title: str = Field(description="Title of the paper")
    abstract: str = Field(description="Abstract of the paper")
    references: list[str] = Field(description="List of titles from the 'references' section")

class ParseLLM:
    def __init__(self, model_name, api_key):
        self.llm = Tongyi(model=model_name, api_key=api_key)
        
        # 使用 PydanticOutputParser 来解析结构化输出
        self.output_parser = PydanticOutputParser(pydantic_object=PaperInfo)
        
        # 解析引用，对应建树方法2
        self.parse_prompt_template = PromptTemplate(template="""
            <document>{document}</document>
            
            Extract the following information from the document above and return it in JSON format:
            - title: Title of the paper
            - abstract: Abstract of the paper
            - references: List of titles from the "references" section (exact strings as they appear). Only extract titles, do not include other information.

            {format_instructions}
        """, input_variables=["document"], partial_variables={"format_instructions": self.output_parser.get_format_instructions()})

    def response(self, prompt):
        try:
            # 尝试调用模型并获取响应
            response = self.llm.invoke(prompt)
            if not response or not isinstance(response, str):
                raise ValueError("Invalid response received from the model.")
            return response
        except Exception as e:
            # 捕获异常并抛出自定义错误信息
            raise RuntimeError(f"Error occurred while invoking the model: {str(e)}")

    def parse_paper(self, paper):
        try:
            # 格式化提示模板
            prompt = self.parse_prompt_template.format(document=paper)
            # 获取模型响应
            response = self.response(prompt)
            
            # 使用 PydanticOutputParser 解析响应为结构化数据
            structured_output = self.output_parser.parse(response)
            return structured_output
        except Exception as e:
            # 捕获异常并抛出自定义错误信息
            raise RuntimeError(f"Error occurred while parsing the paper: {str(e)}")

In [ ]:
# 测试
# 初始化解析器
# parser =ParseLLM(model_name="qwen-turbo",api_key=dashscope_api_key)

# # 输入论文内容
# paper_content = """
# <title>Attention Is All You Need</title>
# <abstract>We propose a new architecture called Transformer...</abstract>
# <references>
# - Neural Machine Translation by Jointly Learning to Align and Translate
# - ImageNet Classification with Deep Convolutional Neural Networks
# </references>
# """

# try:
#     # 解析论文
#     result = parser.parse_paper(paper_content)
    
#     # 输出结果
#     print("Title:", result.title)
#     print("Abstract:", result.abstract)
#     print("References:", result.references)
# except RuntimeError as e:
#     print(f"An error occurred: {e}")

Title: Attention Is All You Need
Abstract: We propose a new architecture called Transformer...
References: ['Neural Machine Translation by Jointly Learning to Align and Translate', 'ImageNet Classification with Deep Convolutional Neural Networks']


## 3.进行建树
1. 获取根节点文献内容，解析，获取摘要和引用文献名
2. 获取引用文献内容，解析论文获取摘要
3. 使用嵌入模型对摘要向量化，选择摘要相关性top k，且之前未出现在溯源树中论文的作为子节点
4. 然后对子节点继续进行步骤1-3，直至溯源树达到要求的层次

辅助函数
- TODO 改用LangGraph处理格式问题、解析为统一的类

In [ ]:
import os
import json
from anytree import Node, RenderTree

# 将模型返回的带有 Markdown 格式代码块的 JSON 字符串解析为 Python 字典
def parse_to_json(response):
    """
    解析模型返回的带有 Markdown 格式代码块的 JSON 字符串。
    
    :param response: 模型返回的字符串，例如：
                     ```json\n{\n  "论文题目": "INJEC AGENT...",\n  "发布时间": "2024-08-04",\n  "摘要": "Recent work..."\n}\n```
    :return: 解析后的 Python 字典
    """
    try:
        # Step 1: 去掉 Markdown 格式的代码块标记
        if response.startswith("```json") and response.endswith("```"):
            # 去掉开头的 ```json 和结尾的 ```
            response = response[len("```json"):-len("```")].strip()
        
        # Step 2: 将剩余的字符串解析为 JSON
        parsed_data = json.loads(response)
        return parsed_data
    except json.JSONDecodeError as e:
        print(f"JSON 解析失败: {e}")
        return None
    except Exception as e:
        print(f"发生错误: {e}")
        return None
    
# 打印树结构
def print_tree(root_node):
    for pre, fill, node in RenderTree(root_node):
        print(f"{pre}{node.name}")
        
# 将树转换为dict
def tree_to_dict(node):
    """
    将 anytree 节点及其子节点递归转换为字典。
    
    参数:0
        node: anytree 节点。
        
    返回:
        表示树结构的字典。
    """
    return {
        "name": node.name,
        "data": node.data,  # 假设每个节点有 data 属性
        "children": [tree_to_dict(child) for child in node.children]
    }

# 将树保存为json
def save_tree_to_json(root, file_path):
    """
    将 anytree 树保存为 JSON 文件。
    
    参数:
        root: 树的根节点。
        file_path: 输出 JSON 文件的路径。
    """
    # 确保文件路径的目录存在
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    # 将树转换为字典
    tree_dict = tree_to_dict(root)
    
    # 写入 JSON 文件
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(tree_dict, f, ensure_ascii=False, indent=4)
    
    print(f"树已成功保存到: {file_path}")
        
# 将字典转换为树
def dict_to_tree(tree_dict, parent=None):
    """
    将字典形式的树结构递归转换为 anytree 节点。
    
    参数:
        tree_dict: 表示树结构的字典。
        parent: 当前节点的父节点（用于递归构建子节点）。
        
    返回:
        构建完成的 anytree 节点。
    """
    # 创建当前节点
    current_node = Node(tree_dict["name"], parent=parent, data=tree_dict.get("data"))
    
    # 递归构建子节点
    for child_dict in tree_dict.get("children", []):
        dict_to_tree(child_dict, parent=current_node)
    
    return current_node

# 从json加载树
def load_tree_from_json(file_path):
    """
    从 JSON 文件中加载树结构并返回根节点。
    
    参数:
        file_path: 输入 JSON 文件的路径。
        
    返回:
        树的根节点。
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        tree_dict = json.load(f)
    
    # 从字典重建树
    root = dict_to_tree(tree_dict)
    return root

In [6]:
import json
from anytree import Node, RenderTree
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document  # 导入 Document 类
from tqdm import tqdm  # 用于显示进度条

# 建树
def parse_and_build_trace_tree(root_paper_title, max_layers, parse_llm, embedding, output_path, top_k):
    """
    构建溯源树。
    
    参数:
        root_paper_title (str): 根论文标题。
        max_layers (int): 最大层数。
        parse_llm: 用于解析论文内容的 LLM 解析器。
        embedding: 用于向量化文本的嵌入模型。
        output_path (str): 输出 JSON 文件路径。
        top_k (int): 每层选择的相似论文数量。
    """
    print(f"开始构建溯源树，根论文: {root_paper_title}")

    # Step 1：处理根论文
    paper_text = search_paper(root_paper_title)
    if not paper_text:
        print(f"无法获取论文: {root_paper_title}")
        return

    paper_info = parse_llm.parse_paper(paper_text)
    json_paper_info = parse_to_json(paper_info)

    # 创建根节点
    root_node = Node(json_paper_info["title"], data=json_paper_info)

    # 哈希表统计当前已经在树中的论文
    in_tree_papers_set = set()
    in_tree_papers_set.add(root_paper_title)

    # 初始化队列，将根论文加入队列
    queue = [(root_node, 0)]  # (当前节点, 当前层数)

    # 初始化内存向量数据库
    vector_store = InMemoryVectorStore(embedding)

    # Step 2：使用队列按层次进行建树
    total_nodes_processed = 0  # 统计处理的节点总数
    with tqdm(total=max_layers, desc="构建溯源树", unit="层") as pbar:
        while queue:
            current_node, current_layer = queue.pop(0)

            # 如果达到最大层数，停止扩展
            if current_layer >= max_layers:
                break

            print(f"\n正在处理第 {current_layer + 1} 层，当前节点: {current_node.name}")

            # 获取当前论文的引用论文内容
            references = current_node.data.get("references", [])
            if not references:
                print("当前节点没有引用论文，跳过...")
                continue

            print(f"正在解析 {len(references)} 篇引用论文...")

            # 解析引用论文的内容并提取摘要
            reference_docs = []
            for ref_title in references:
                print(f"正在获取引用论文: {ref_title}")
                ref_text = search_paper(ref_title)
                if not ref_text:
                    continue

                ref_info = parse_llm.parse_paper(ref_text)
                ref_json_info = parse_to_json(ref_info)

                # 将引用论文的摘要和题目转为 Document 类型，添加到 vector_store 中
                if ref_json_info is None:
                    continue
                
                doc = Document(page_content=ref_json_info["abstract"], metadata={"title": ref_json_info["title"]})
                vector_store.add_documents([doc])
                reference_docs.append(doc)

            # 使用当前论文的摘要作为匹配文本，在 vector_store 中进行相似度匹配
            current_abstract = current_node.data["abstract"]
            similar_docs = vector_store.similarity_search(current_abstract, k=top_k)

            # 筛选未出现在树中的论文
            for doc in similar_docs:
                title = doc.metadata["title"]
                if title not in in_tree_papers_set:
                    # 获取论文内容
                    paper_text = search_paper(title)
                    # 解析论文内容
                    try:
                        paper_info = parse_llm.parse_paper(paper_text)
                        json_paper_info = parse_to_json(paper_info)  # 转换为 JSON 格式
                    except Exception as e:
                        print(f"解析论文失败: {title}，错误信息: {e}")
                        continue
                    
                    # 创建新节点并加入树中
                    new_node = Node(title, parent=current_node, data=json_paper_info)
                    in_tree_papers_set.add(title)

                    # 将新节点加入队列
                    queue.append((new_node, current_layer + 1))


            # 处理完一个节点，清空 vector_store
            vector_store = InMemoryVectorStore(embedding)

            # 更新进度条
            total_nodes_processed += 1
            pbar.update(1)

    # Step 3：后处理，打印、存储
    print("\n溯源树构建完成！最终结构如下：")
    print_tree(root_node)

    # 存储
    save_tree_to_json(root_node, output_path)
    print(f"溯源树已保存为 JSON 文件: {output_path}")

In [7]:
# embedding
from langchain_community.embeddings import DashScopeEmbeddings
# a simple, in memory vector store
from langchain_core.vectorstores import InMemoryVectorStore

parse_llm=ParseLLM(model_name="qwen-turbo",api_key=dashscope_api_key)

embeddings = DashScopeEmbeddings(
    model="text-embedding-v3", dashscope_api_key=dashscope_api_key
)

In [ ]:
# 进行测试
parse_and_build_trace_tree(
    root_paper_title="LightRAG",
    max_layers=3,
    parse_llm=parse_llm,
    embedding=embeddings,
    output_path="output/trace_tree.json",
    top_k=3
)

## 4.绘制溯源树
尽量进行美化

In [ ]:
# TODO
